## reuters 
The Reuters Corpus contains 10,788 news documents totaling 1.3 million words. The documents have been classified into 90 topics, and grouped into two sets, called "training" and "test"; thus, the text with fileid 'test/14826' is a document drawn from the test set. This split is for training and testing algorithms that automatically detect the topic of a document, as we will see in chap-data-intensive.

In [1]:
from nltk.corpus import reuters
# len(reuters.fileids())
# reuters.fileids() #gets each file

In [2]:
type(reuters.words('test/14826'))

# string_list

nltk.corpus.reader.util.StreamBackedCorpusView

#### calculating no. of grams

In [3]:
from collections import defaultdict
from nltk import ngrams
from funcy import flatten, isa

unigram=defaultdict(int)
bigram=defaultdict(int)
trigram=defaultdict(int)
fourgram=defaultdict(int)
fivegram=defaultdict(int)

def gram_calc(sentence,gram):
    if gram==1:
        temp=list(ngrams(sentence,gram))
        for gram in temp:
            unigram[gram]+=1
        return unigram
    elif gram==2:
        temp=list(ngrams(sentence,gram))
        for gram in temp:
            bigram[gram]+=1
        return bigram
    elif gram==3:
        temp=list(ngrams(sentence,gram))
        for gram in temp:
            trigram[gram]+=1
        return trigram
    elif gram==4:
        temp=list(ngrams(sentence,gram))
        for gram in temp:
            fourgram[gram]+=1
        return fourgram
    elif gram==5:
        temp=list(ngrams(sentence,gram))
        for gram in temp:
            fivegram[gram]+=1
        return fivegram

def start_stop_adder(news):
    sentence=[]
    temp=[]
    for word in news:
        # print(word)
        if word!=".":
            temp.append(word)
        elif word==" ":
            continue
        else:
            try:
                temp[0]="<s>"
                temp[len(temp)-1]="</s>"
            except IndexError:
                pass
            sentence.append([" ".join(temp)])
            temp=[]
    return sentence

for news in reuters.fileids():
    lowered_news = [each_string.lower() for each_string in reuters.words(news)]
    sentences=start_stop_adder(lowered_news)
    para=[]
    for sentence in sentences:
        para.append(sentence[0].split(" "))
    para = list(flatten(para))
    gram_calc(para,1)
    gram_calc(para,2)
    gram_calc(para,3)
    gram_calc(para,4)
    gram_calc(para,5)
    

#### calculating probabilities

In [4]:
uni_count=0
for key,value in unigram.items():
    uni_count+=value
print(uni_count)

bi_count=0
for key,value in bigram.items():
    bi_count+=value
print(bi_count)

tri_count=0
for key,value in trigram.items():
    tri_count+=value
print(tri_count)

four_count=0
for key,value in fourgram.items():
    four_count+=value
print(four_count)

five_count=0
for key,value in fivegram.items():
    five_count+=value
print(five_count)

1569202
1559452
1549702
1539961
1530223


#### conditional probability : <br>
Bigram   : p(b|a)=p(a∩b)/p(a)<br>
trigram  : P(A| B,C) =P(A∩B∩C) /P(B∩C)<br>
fourgram : P(A| B,C,D) =P(A∩B∩C∩D) /P(B∩C∩D)<br>
fivegram : P(A| B,C,D,E) =P(A∩B∩C∩D∩E) /P(B∩C∩D∩E)<br>

In [5]:
uniprob=defaultdict(int)
for key, value in unigram.items():
    new_value = value/uni_count
    uniprob[key] = new_value

In [6]:
biprob=defaultdict(int)
for key, value in bigram.items():
    temp=[]
    temp.append(f'{key[1]}')
    temp=tuple(temp)
    prob_val=value/unigram[temp]
    biprob[key]=prob_val 
 

In [7]:
triprob=defaultdict(int)
for key, value in trigram.items():
    temp=key[1:3]
    prob_val=value/bigram[temp]
    triprob[key]=prob_val 

In [19]:
fourprob=defaultdict(int)
for key,value in fourgram.items():
    temp=key[1:4]
    prob_val=value/trigram[temp]
    fourprob[key]=prob_val
# four_prob

In [20]:
fiveprob=defaultdict(int)
for key,value in fivegram.items():
    temp=key[1:5]
    prob_val=value/fourgram[temp]
    fiveprob[key]=prob_val
# four_prob

In [10]:
# biprob

#### using markov's assumption on chain rule of probability


#### prediction
n = how many words has to be predicted<br>
grams=how many grams does user wants us to predict the next word<br>

In [49]:
def predict(ip,n,grams):
    ip=ip.split(" ")
    if grams==2:
        for i in range(n):
            # print(ip)
            temp=[0]
            predictor=0.0
            for key,value in biprob.items():
                try:
                    if key[0]==ip[-1]:
                        if biprob[key]>predictor:
                            predictor=biprob[key]
                            temp[0]=(key,value)
                except TypeError:
                    pass
            # print(temp)
            # print(ip)
            ip.append(temp[0][0][-1])
        # print(ip)
        # print(temp)
        return " ".join(ip)
    if grams==3:
        for i in range(n):
            # print(ip)
            temp=[0]
            predictor=0.0
            for key,value in triprob.items():
                try:
                    if key[0:2]==tuple(ip[-2:]):
                        # print("here",key[0:2],ip[-2:])
                        if triprob[key]>predictor:
                            predictor=triprob[key]
                            temp[0]=(key,value)
                except TypeError:
                    pass
            # print("temp",temp)
            # print(temp[0][0][-1])
            # print(ip)
            # break
            # print([0][0])
            ip.append(temp[0][0][-1])
        # print("after append ip",ip)
        # print("temp",temp)
        return " ".join(ip)
    if grams==4:
        for i in range(n):
            # print(ip)
            temp=[0]
            predictor=0.0
            for key,value in fourprob.items():
                try:
                    if key[0:3]==tuple(ip[-3:]):
                        # print("here",key[0:2],ip[-2:])
                        if fourprob[key]>predictor:
                            predictor=fourprob[key]
                            temp[0]=(key,value)
                except TypeError:
                    pass
            # print("temp",temp)
            # print(temp[0][0][-1])
            # print(ip)
            # break
            # print([0][0])
            ip.append(temp[0][0][-1])
        # print("after append ip",ip)
        # print("temp",temp)
        return " ".join(ip)
    if grams==5:
        for i in range(n):
            # print(ip)
            temp=[0]
            predictor=0.0
            for key,value in fiveprob.items():
                try:
                    if key[0:4]==tuple(ip[-4:]):
                        # print("here",key[0:2],ip[-2:])
                        if fiveprob[key]>predictor:
                            predictor=fiveprob[key]
                            temp[0]=(key,value)
                except TypeError:
                    pass
            # print("temp",temp)
            # print(temp[0][0][-1])
            # print(ip)
            # break
            # print([0][0])
            ip.append(temp[0][0][-1])
        # print("after append ip",ip)
        # print("temp",temp)
        return " ".join(ip)
 
 
 
print(predict("exporters fear",3,2)) #predicting the next two words using bigram
print(predict("exporters fear damage",5,3))#predicting the next five words using trigram
print(predict("japan has raised fears",2,4))#predicting the next five words using fourgram
print(predict("raised fears among many of",6,5))#predicting the next six words using fivegrams

exporters fear tighter monetary gamesmanship
exporters fear damage from frigid temperatures expected tonight
japan has raised fears among many
raised fears among many of asia ' s exporting nations that
